In [1]:
import pytorch_lightning as pl
from src import data_loaders
from src.models import resnet_18
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
from pytorch_lightning.loggers import WandbLogger

In [2]:
wandb_logger = WandbLogger(project="Resnet18_190", entity="agrindro")
model = resnet_18.ResNetLightning()

train_loader, val_loader = data_loaders.cifar10_loader(train=True, num_workers=4)
test_loader = data_loaders.cifar10_loader(train=False, num_workers=4)

callbacks = [ModelCheckpoint(filename="/kaggle/working/models/best", monitor='val_accuracy', mode='max', save_top_k=1, verbose=True, every_n_epochs=1),
            RichProgressBar()
            ]

trainer = pl.Trainer(logger=wandb_logger, callbacks=callbacks, max_epochs=10)

Files already downloaded and verified
Files already downloaded and verified


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [3]:
trainer.fit(model, train_loader, val_loader)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alexggrindrod (agrindro). Use `wandb login --relogin` to force relogin



  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]